In [1]:
##Install Prerequesites
#!pip install -q git+https://github.com/huggingface/transformers.git
#!pip install -q timm

In [2]:
import torch
from huggingface_hub import hf_hub_download
from PIL import Image
from transformers import DetrFeatureExtractor
from transformers import TableTransformerForObjectDetection

In [3]:
def scaler(box,xscale,yscale,xmax,ymax):
    box[0]*=1-xscale
    box[1]*=1-yscale
    box[2]*=1+xscale
    box[3]*=1+yscale
    if box[2]>xmax:
        box[2] = xmax
    if box[3]>ymax:
        box[3] = ymax
    return box

In [4]:
def table_detection(image):
    width, height = image.size
    feature_extractor = DetrFeatureExtractor()
    encoding = feature_extractor(image, return_tensors="pt")
    model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection")

    with torch.no_grad():
        outputs = model(**encoding)

    results = feature_extractor.post_process_object_detection(outputs, threshold=0.7, target_sizes=[(height, width)])[0]

    cropped_img = []
    scale = 0.05
    for i in range(len(results['scores'])):
        bounding_box = scaler(results['boxes'][i].tolist(),scale,scale,width,height)
        cropped_img.append(image.crop(bounding_box))
    
    return cropped_img

In [5]:
def table_structure_detection(image):
    width, height = image.size
    feature_extractor = DetrFeatureExtractor()
    encoding = feature_extractor(image, return_tensors="pt")
    model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-structure-recognition")
    
    with torch.no_grad():
        outputs = model(**encoding)
    target_sizes = [image.size[::-1]]
    results = feature_extractor.post_process_object_detection(outputs, threshold=0.8, target_sizes=target_sizes)[0]
    
    output = {'col_data':[],'row_data':[],'col_header_data':[],'row_header_data':[]}
    
    #plot_results(model,image, results['scores'], results['labels'], results['boxes'])
    y_scale = 0.03
    for i in range(len(results['boxes'])):
        if results['labels'][i] == 1:
            bounding_box = scaler(results['boxes'][i].tolist(),1,y_scale,width,height)
            output['col_data'].append(image.crop(bounding_box))
        elif results['labels'][i] == 2:
            bounding_box = scaler(results['boxes'][i].tolist(),1,y_scale,width,height)
            output['row_data'].append(image.crop(bounding_box))
        elif results['labels'][i] == 3:
            bounding_box = scaler(results['boxes'][i].tolist(),1,y_scale,width,height)
            output['col_header_data'].append(image.crop(bounding_box))
        elif results['labels'][i] == 4:
            bounding_box = scaler(results['boxes'][i].tolist(),1,y_scale,width,height)
            output['row_header_data'].append(image.crop(bounding_box))
    
    return output

In [6]:
import matplotlib.pyplot as plt

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def plot_results(model,pil_img, scores, labels, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{model.config.id2label[label]}: {score:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [48]:
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import os # folder directory navigation
from numpy import asarray

ocr_model = PaddleOCR(lang='en')

def extract_data(split_table):
    output = []
    for i in split_table:
        #Ran OCR to get a split list output
        #output.append(ocr_model.ocr(i))
        numpydata = asarray(i)
        output.append(ocr_model.ocr(numpydata, cls=True))

    return output

[2023/06/13 14:15:30] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/jovyan/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/jovyan/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_pa

In [ ]:
def get_table_data(img):
    #Locate tables on the image
    tables = table_detection(img)
    #Figure out headers and data inside the table
    split_tables = {}
    for i,table in enumerate(tables):
        split_tables[i] = table_structure_detection(table)
    
    
    #Perform OCR on each table detected
    data = {}
    for i,split_table in split_tables.items():
        if len(split_table['col_header_data'])>0:
            data[i]['header_data'] = extract_data(split_table['col_header_data'])
        if len(split_table['row_data'])>0:
            data[i]['row_data'] = extract_data(split_table['row_data'])
    return data

In [49]:
file_path = "/data/invoice_sample.jpg"
image = Image.open(file_path).convert("RGB")
    
data = get_table_data(image)
print(data)

[2023/06/13 14:15:36] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/06/13 14:15:36] ppocr DEBUG: dt_boxes num : 5, elapse : 0.013402938842773438
[2023/06/13 14:15:36] ppocr DEBUG: rec_res num  : 5, elapse : 0.15654873847961426
[2023/06/13 14:15:36] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/06/13 14:15:36] ppocr DEBUG: dt_boxes num : 5, elapse : 0.031442880630493164
[2023/06/13 14:15:37] ppocr DEBUG: rec_res num  : 5, elapse : 0.5468142032623291
[2023/06/13 14:15:37] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/06/13 14:15:37] ppocr DEBUG: dt_boxes num : 6, elapse : 0.035529136657714844
[2023/06/13 14:15:37] ppocr DEBUG: rec_res num  : 6, elapse : 0.23351407051086426
[2023/06/13 14:15:37] ppocr WARNING: Since the angle classi